In [1]:
from coding_agents import run_to_clipboard

# a = 'write a function that extract all the function from a python file to an output file named functions.py' 
# b = 'write a function that get a string an execute it in the running environment '
# a = 'write a function that get a python file print all the names of the functions and extract all the functions as a dictionary with: original function, function header, return and arguments'
# a = 'write a python file with a gardio ui that have a chatbot window on the left and a window on the left that show a python file with sytax highlight '
# a = 'write a function that get a backup folder named output and a format ca[i]_prompt.txt and return the largest i'
a = 'write a function that get a code string and run it in the original running environment'
print(run_to_clipboard(a))

[{'original': 'def run_code_in_context(code_str):\n    try:\n        # Get the caller\'s frame\n        caller_frame = inspect.currentframe().f_back\n        \n        # Get caller\'s global and local variables\n        caller_globals = caller_frame.f_globals\n        caller_locals = caller_frame.f_locals\n        \n        # Execute code in caller\'s context\n        exec(code_str, caller_globals, caller_locals)\n        \n    except SyntaxError as e:\n        print(f"Syntax error in code: {e}")\n        return None\n    except Exception as e:\n        print(f"Error executing code: {e}")\n        return None\n    finally:\n        # Clean up frame reference\n        del caller_frame', 'header': 'def run_code_in_context(code_str):', 'arguments': ['code_str'], 'returns': ['return None', 'return None']}]

import inspect
import sys

def run_code_in_context(code_str):
    try:
        # Get the caller's frame
        caller_frame = inspect.currentframe().f_back
        
        # Get calle

In [2]:
import requests
import json

def handle_sources_and_answer(question: str, base_url: str = "http://localhost:8000") -> str:
    """
    Send a question to get sources and then get an answer from the backend server.
    
    Args:
        question: The research question to ask
        base_url: The base URL of the backend server (default: http://localhost:8000)
        
    Returns:
        str: The answer from the backend
    """
    try:
        # First get the sources
        sources_url = f"{base_url}/api/getSources"
        sources_response = requests.post(
            sources_url,
            data=json.dumps({"question": question}),
            headers={"Content-Type": "application/json"}
        )
        sources_response.raise_for_status()
        sources = sources_response.json()

        # Then get the answer using the sources
        answer_url = f"{base_url}/api/getAnswer"
        answer_response = requests.post(
            answer_url,
            json={"question": question, "sources": sources},
            headers={"Content-Type": "application/json"}
        )
        answer_response.raise_for_status()

        # Handle 202 status specifically
        if answer_response.status_code == 202:
            return answer_response.text

        # Handle streaming response if needed
        if answer_response.headers.get('transfer-encoding') == 'chunked':
            full_response = ""
            for chunk in answer_response.iter_content(chunk_size=1024):
                if chunk:
                    full_response += chunk.decode('utf-8')
            return full_response
            
        return answer_response.text

    except requests.RequestException as e:
        raise Exception(f"Error communicating with the server: {str(e)}")

# Example usage
if __name__ == "__main__":
    try:
        question = "What are the latest developments in AI?"
        answer = handle_sources_and_answer(question)
        print("Answer:", answer)
    except Exception as e:
        print(f"Error: {str(e)}")

Error: Error communicating with the server: 404 Client Error: Not Found for url: http://localhost:8000/api/getSources
